In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Conv1D, MaxPooling1D, Flatten, Input

from statsmodels.tsa.arima.model import ARIMA

In [67]:
df = pd.read_csv("dengue-dataset-with-YMD.csv")
df.columns = df.columns.str.strip()

WINDOW = 12        # sequence length
TRAIN_RATIO = 0.8  # train/test split

In [68]:
percentiles_per_city = (
    df.groupby("CITY")["INCIDENCE_per_100k"]
    .quantile([0.50, 0.75, 0.95])
    .unstack()
    .rename(columns={0.50: "P50", 0.75: "P75", 0.95: "P95"})
)

def risk_from_cases(city, cases):
    p50 = percentiles_per_city.loc[city, "P50"]
    p75 = percentiles_per_city.loc[city, "P75"]
    p95 = percentiles_per_city.loc[city, "P95"]
    if cases <= p50:
        return "Low"
    elif cases <= p75:
        return "Moderate"
    elif cases <= p95:
        return "High"
    else:
        return "Very High"

In [69]:
def create_window_dataset(series, window):
    X, y = [], []
    for i in range(len(series) - window):
        X.append(series[i:i+window])
        y.append(series[i+window])
    return np.array(X), np.array(y)

In [70]:
def lstm_forecast(X_train, y_train, X_test, y_test, window=WINDOW, epochs=30, batch_size=16):
    model = Sequential([
        Input(shape=(window,1)),
        LSTM(64, activation="tanh"),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=0)
    return model.predict(X_test)

In [71]:
def gru_forecast(X_train, y_train, X_test, y_test, window=WINDOW, epochs=30, batch_size=16):
    model = Sequential([
        Input(shape=(window,1)),
        GRU(64, activation="tanh"),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=0)
    return model.predict(X_test)

In [72]:
def cnn_lstm_forecast(X_train, y_train, X_test, y_test, window=WINDOW, epochs=30, batch_size=16):
    model = Sequential([
        Input(shape=(window,1)),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(64, activation='tanh'),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=0)
    return model.predict(X_test)

In [73]:
def arima_forecast(series_train, series_test, order=(1,0,0)):
    model = ARIMA(series_train, order=order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(series_test))
    return forecast.reshape(-1,1)

In [74]:
def arima_lstm_hybrid(series_train, series_test, window=WINDOW, epochs=30, batch_size=16, arima_order=(1,0,0)):
    # 1. ARIMA predicts linear part
    arima_model = ARIMA(series_train, order=arima_order).fit()
    arima_pred = arima_model.forecast(steps=len(series_test)).reshape(-1,1)

    # 2. Residual = actual - ARIMA prediction
    residual_train = series_train[-len(series_train):] - arima_model.fittedvalues[-len(series_train):]
    scaler = MinMaxScaler()
    residual_scaled = scaler.fit_transform(residual_train.reshape(-1,1))

    # 3. LSTM on residual
    X_res, y_res = create_window_dataset(residual_scaled, window)
    X_train_res, X_test_res = X_res[:-len(series_test)], X_res[-len(series_test):]
    y_train_res, y_test_res = y_res[:-len(series_test)], y_res[-len(series_test):]

    lstm_model = Sequential([
        Input(shape=(window,1)),
        LSTM(64, activation="tanh"),
        Dense(1)
    ])
    lstm_model.compile(optimizer="adam", loss="mse")
    lstm_model.fit(X_train_res, y_train_res, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=0)

    lstm_pred = lstm_model.predict(X_test_res)
    lstm_pred_inverse = scaler.inverse_transform(lstm_pred)

    # 4. Hybrid = ARIMA + LSTM residual
    hybrid_pred = arima_pred + lstm_pred_inverse
    return hybrid_pred

In [75]:
def evaluate_predictions(city, y_true_cases, pred_cases):
    # Risk mapping
    pred_risk = np.array([risk_from_cases(city, c[0]) for c in pred_cases])
    y_true_risk = np.array([risk_from_cases(city, c[0]) for c in y_true_cases])

    # Metrics
    rmse = np.sqrt(mean_squared_error(y_true_cases, pred_cases))
    mae = mean_absolute_error(y_true_cases, pred_cases)

    nonzero_idx = y_true_cases.flatten() != 0
    mape = np.mean(np.abs((y_true_cases.flatten()[nonzero_idx] - pred_cases.flatten()[nonzero_idx]) / y_true_cases.flatten()[nonzero_idx])) * 100

    acc = accuracy_score(y_true_risk, pred_risk)
    f1 = f1_score(y_true_risk, pred_risk, average='weighted')

    return {"RMSE": rmse, "MAE": mae, "MAPE": mape, "Risk_Accuracy": acc, "Risk_F1": f1}

In [76]:
algorithms = {
    "LSTM": lstm_forecast,
    "GRU": gru_forecast,
    "CNN-LSTM": cnn_lstm_forecast,
    "ARIMA": arima_forecast,
    "Hybrid": arima_lstm_hybrid
}

results = []

for city in df["CITY"].unique():
    data = df[df["CITY"] == city].copy()
    target = data["CASES"].values.reshape(-1, 1)

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(target)

    X, y = create_window_dataset(scaled, WINDOW)
    split = int(len(X) * TRAIN_RATIO)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    city_metrics = {"CITY": city}

    for algo_name, algo_func in algorithms.items():
        # ------------------------------
        # 1. Model-specific predictions
        # ------------------------------
        if algo_name in ["ARIMA", "Hybrid"]:
            pred_cases = algo_func(target[:split], target[split:])
            pred_cases = np.array(pred_cases).reshape(-1, 1)  # ensure shape
        else:
            pred_scaled = algo_func(X_train, y_train, X_test, y_test)
            pred_cases = scaler.inverse_transform(pred_scaled)

        # ---------------------------------------
        # 2. Inverse transform ground truth cases
        # ---------------------------------------
        y_true_cases = scaler.inverse_transform(y_test)

        # ---------------------------------------------
        # 3. FIX LENGTH MISMATCH FOR ALL ALGORITHMS HERE
        # ---------------------------------------------
        min_len = min(len(pred_cases), len(y_true_cases))
        pred_cases = pred_cases[:min_len]
        y_true_cases = y_true_cases[:min_len]

        # Evaluate & store results
        metrics = evaluate_predictions(city, y_true_cases, pred_cases)
        for k, v in metrics.items():
            city_metrics[f"{algo_name}_{k}"] = v

    results.append(city_metrics)

metrics_df = pd.DataFrame(results)
metrics_df

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 

,CITY,LSTM_RMSE,LSTM_MAE,LSTM_MAPE,LSTM_Risk_Accuracy,LSTM_Risk_F1,GRU_RMSE,GRU_MAE,GRU_MAPE,GRU_Risk_Accuracy,...,ARIMA_RMSE,ARIMA_MAE,ARIMA_MAPE,ARIMA_Risk_Accuracy,ARIMA_Risk_F1,Hybrid_RMSE,Hybrid_MAE,Hybrid_MAPE,Hybrid_Risk_Accuracy,Hybrid_Risk_F1
0,CALOOCAN CITY,14.386070,12.218534,204.244092,0.40,0.228571,11.474202,8.612824,132.926259,0.44,...,50.120275,49.164450,984.423167,0.40,0.228571,96.298571,91.195664,1809.751705,0.40,0.228571
1,LAS PINAS CITY,7.212480,4.410389,128.329027,0.32,0.368932,6.176840,4.182376,127.385103,0.30,...,19.820003,19.231299,902.334457,0.16,0.044138,52.280879,51.506569,2381.049483,0.16,0.044138
2,MAKATI CITY,5.098380,4.096140,114.071833,0.32,0.239102,4.200463,3.256608,82.016445,0.38,...,21.838504,20.577793,711.517880,0.26,0.107302,74.958736,74.662438,2686.317318,0.26,0.107302
3,MALABON CITY,4.610802,3.365570,75.537099,0.56,0.538820,6.507766,5.018628,141.209777,0.44,...,53.231710,50.333590,1724.707005,0.06,0.006792,146.812722,146.031337,5309.630111,0.06,0.006792
4,MANDALUYONG CITY,5.326330,4.579822,197.873683,0.12,0.047955,5.571958,3.716326,133.382994,0.18,...,9.656599,9.400691,441.132074,0.04,0.003077,9.236653,8.142455,366.605345,0.20,0.243556
5,MANILA CITY,13.155243,12.215053,415.235644,0.38,0.209275,8.877907,7.848870,231.290505,0.38,...,59.371487,58.636627,2060.585223,0.38,0.209275,162.176273,157.875772,5317.679474,0.38,0.209275
6,MARIKINA CITY,4.307253,3.483448,142.653591,0.36,0.258333,3.690998,2.631506,99.433639,0.40,...,14.638536,14.311532,679.681302,0.14,0.034386,44.503014,42.897886,2063.293471,0.14,0.034386
7,MUNTINLUPA CITY,4.146476,3.147164,139.229895,0.30,0.317942,2.861815,1.983472,116.269139,0.72,...,30.558653,28.822601,1342.442919,0.10,0.018182,87.629616,87.126396,4308.058574,0.10,0.018182
8,NAVOTAS CITY,3.169173,2.601627,84.223491,0.34,0.360859,3.028282,2.309695,67.002569,0.40,...,6.050288,5.740422,220.636494,0.16,0.124167,16.448645,15.283118,786.845659,0.12,0.030545
9,PARANAQUE CITY,6.910177,5.143442,154.706574,0.46,0.444389,9.905175,8.648905,294.897508,0.22,...,62.065889,59.489739,2297.033796,0.14,0.034386,152.035390,150.395952,5308.747201,0.14,0.034386


In [77]:
print(metrics_df)

                CITY  LSTM_RMSE   LSTM_MAE   LSTM_MAPE  LSTM_Risk_Accuracy  \
0      CALOOCAN CITY  14.386070  12.218534  204.244092                0.40   
1     LAS PINAS CITY   7.212480   4.410389  128.329027                0.32   
2        MAKATI CITY   5.098380   4.096140  114.071833                0.32   
3       MALABON CITY   4.610802   3.365570   75.537099                0.56   
4   MANDALUYONG CITY   5.326330   4.579822  197.873683                0.12   
5        MANILA CITY  13.155243  12.215053  415.235644                0.38   
6      MARIKINA CITY   4.307253   3.483448  142.653591                0.36   
7    MUNTINLUPA CITY   4.146476   3.147164  139.229895                0.30   
8       NAVOTAS CITY   3.169173   2.601627   84.223491                0.34   
9     PARANAQUE CITY   6.910177   5.143442  154.706574                0.46   
10        PASAY CITY   3.547947   3.095573  153.510853                0.06   
11        PASIG CITY   8.680747   6.576271  180.966104          

In [79]:
import pandas as pd

# Assuming metrics_df is already defined
df = metrics_df.copy()

# 1️⃣ Overview of all algorithms and metrics
algorithms = ["LSTM", "GRU", "CNN-LSTM", "ARIMA", "Hybrid"]
metrics = ["RMSE", "MAE", "MAPE", "Risk_Accuracy", "Risk_F1"]

# 2️⃣ Best and worst performance per algorithm
best_worst = {}
for algo in algorithms:
    best_worst[algo] = {}
    for metric in metrics:
        col = f"{algo}_{metric}"
        best_val = df[col].min()
        worst_val = df[col].max()
        best_city = df.loc[df[col] == best_val, "CITY"].values[0]
        worst_city = df.loc[df[col] == worst_val, "CITY"].values[0]
        best_worst[algo][metric] = {"best_city": best_city, "best_value": best_val,
                                    "worst_city": worst_city, "worst_value": worst_val}

# 3️⃣ Average metrics per algorithm across all cities
avg_metrics = {}
for algo in algorithms:
    avg_metrics[algo] = {}
    for metric in metrics:
        col = f"{algo}_{metric}"
        avg_metrics[algo][metric] = df[col].mean()

# 4️⃣ Rankings per city per metric
rankings = {}
for metric in metrics:
    rankings[metric] = df[[f"{algo}_{metric}" for algo in algorithms]]
    rankings[metric] = rankings[metric].rank(axis=1, method='min')  # lower is better for RMSE, MAE, MAPE

# 5️⃣ Identify the overall best algorithm by average metric
overall_best = {}
for metric in metrics:
    best_algo = min(avg_metrics, key=lambda x: avg_metrics[x][metric] if metric != "Risk_Accuracy" else -avg_metrics[x][metric])
    overall_best[metric] = best_algo

# 6️⃣ Optional: Top 3 algorithms per city for RMSE
top3_per_city = {}
for i, row in df.iterrows():
    city = row["CITY"]
    city_ranks = {algo: row[f"{algo}_RMSE"] for algo in algorithms}
    sorted_algos = sorted(city_ranks.items(), key=lambda x: x[1])
    top3_per_city[city] = sorted_algos[:3]

# 7️⃣ Display all results
print("===== Best and Worst per Algorithm =====")
print(pd.DataFrame(best_worst))
print("\n===== Average Metrics per Algorithm =====")
print(pd.DataFrame(avg_metrics))
print("\n===== Overall Best Algorithm by Metric =====")
print(overall_best)
print("\n===== Top 3 Algorithms per City by RMSE =====")
for city, algos in top3_per_city.items():
    print(city, algos)


===== Best and Worst per Algorithm =====
                                                            LSTM  \
RMSE           {'best_city': 'SAN JUAN CITY', 'best_value': 1...   
MAE            {'best_city': 'SAN JUAN CITY', 'best_value': 1...   
MAPE           {'best_city': 'MALABON CITY', 'best_value': 75...   
Risk_Accuracy  {'best_city': 'PASAY CITY', 'best_value': 0.06...   
Risk_F1        {'best_city': 'PASAY CITY', 'best_value': 0.02...   

                                                             GRU  \
RMSE           {'best_city': 'SAN JUAN CITY', 'best_value': 2...   
MAE            {'best_city': 'PASAY CITY', 'best_value': 1.79...   
MAPE           {'best_city': 'NAVOTAS CITY', 'best_value': 67...   
Risk_Accuracy  {'best_city': 'PATEROS', 'best_value': 0.06, '...   
Risk_F1        {'best_city': 'PATEROS', 'best_value': 0.0144,...   

                                                        CNN-LSTM  \
RMSE           {'best_city': 'SAN JUAN CITY', 'best_value': 1...   
MAE  

In [80]:
import os
import pickle
import pandas as pd
from keras.models import save_model

# --- Create directories for saving if they don't exist ---
os.makedirs("saved_models", exist_ok=True)
os.makedirs("saved_data", exist_ok=True)

# --- 1. Save metrics_df ---
metrics_df.to_pickle("saved_data/metrics_df.pkl")
metrics_df.to_csv("saved_data/metrics_df.csv", index=False)  # optional CSV

print("✅ metrics_df saved (pickle & csv)")

# --- 2. Save all predictions per city & algorithm ---
# Assuming you have a dictionary like this
# all_preds = {city: {algo_name: pred_cases_array}}
if 'all_preds' in globals():
    with open("saved_data/all_predictions.pkl", "wb") as f:
        pickle.dump(all_preds, f)
    print("✅ All predictions saved (pickle)")
else:
    print("⚠️ Warning: 'all_preds' dictionary not found, skipping predictions save")

# --- 3. Save Keras models ---
# Assuming your algorithms dict contains model functions and you kept trained models in a dict
if 'trained_models' in globals():
    for algo_name, model in trained_models.items():
        model_path_h5 = f"saved_models/{algo_name}.h5"
        model.save(model_path_h5)
        print(f"✅ Saved {algo_name} model as {model_path_h5}")
else:
    print("⚠️ Warning: 'trained_models' dict not found, skipping model save")

# --- 4. Optional: save as npy if needed ---
np.save("saved_data/metrics_df.npy", metrics_df.values)
print("✅ metrics_df also saved as NumPy array (metrics_df.npy)")

print("\nAll available data saved. You can reload metrics, predictions, and models anytime.")


✅ metrics_df saved (pickle & csv)
⚠️ Warning: 'all_preds' dictionary not found, skipping predictions save
⚠️ Warning: 'trained_models' dict not found, skipping model save
✅ metrics_df also saved as NumPy array (metrics_df.npy)

All available data saved. You can reload metrics, predictions, and models anytime.
